### Import

In [ ]:
import requests
import json
import pandas as pd
from google.colab import files

### Setup

In [ ]:
headers = {'accept': 'application/json'}
key = 'xkks-g1YtxL9-kRoJAGR0_CqsiVcobLR'

In [ ]:
addresses = pd.read_csv('dappradar.csv')

In [ ]:
addresses

,Collection,Symbol,Transaction
0,Milady Maker\n,MIL,0x5Af0D9827E0c53E4799BB226655A1de152A425a5
1,Sappy Seals\n,SAPS,0x364C828eE171616a39897688A831c2499aD972ec
2,The Potatoz\n\n,Potatoz,0x39ee2c7b3cb80254225884ca001F57118C8f21B6
3,GalacticApes\n,GALAPE,0x12d2D1beD91c24f878F37E66bd829Ce7197e4d14
4,MutantCats\n,MUTCATS,0xaAdBA140Ae5e4c8a9eF0Cc86EA3124b446e3E46A
5,MURI by Haus\n,MURI,0x4B61413D4392c806E6d0fF5Ee91E6073C21d6430
6,Nakamigos\n,NKMGS,0xd774557b647330C91Bf44cfEAB205095f7E6c367
7,Arcade Land\n,ARCLAND,0x4a8C9D751EEAbc5521A68FB080DD7E72E46462aF
8,My Pet Hooligan\n,MPH,0x09233d553058c2F42ba751C87816a8E9FaE7Ef10
9,MetaHero Universe: Generative Identities\n,HERO,0x6dc6001535e15b9def7b0f6A20a2111dFA9454E2


In [ ]:
contractNames = addresses['Symbol'].values
contractAddresses = addresses['Transaction'].values

In [ ]:
def do_request(contractAddress, contractName):
    fromBlock = '0'
    toBlock = 'latest'
    pageKey = ''
    sales = []
    initial = True
    
    
    while True:
        if initial:
            url = ('https://eth-mainnet.g.alchemy.com/nft/v2/'+ key +
                   '/getNFTSales?fromBlock=' + fromBlock + '&toBlock=' + toBlock +
                   '&order=asc&contractAddress=' + contractAddress)
        else:
            with open(contractName + '.json', "r") as file:
              sales = json.load(file)
            
            url = ('https://eth-mainnet.g.alchemy.com/nft/v2/'+ key +
                   '/getNFTSales?fromBlock=' + fromBlock + '&toBlock=' + toBlock +
                   '&order=asc&contractAddress=' + contractAddress + '&pageKey=' + pageKey)
            
        response = requests.get(url, headers=headers).text
        
        nftSales = json.loads(response)['nftSales']
        sales += nftSales

        with open(contractName + '.json', "w") as file:
            json.dump(sales, file, indent=2) 
            
        print(len(nftSales))
        
        try:
            pageKey = json.loads(response)['pageKey']
            print(pageKey)
        except:
            print('Pages ended.')
            return sales
        initial = False
   
        if len(nftSales) != 1000:
            return sales

In [ ]:
for i in range(len(contractAddresses)):
    print('----------------------------------')
    print(contractNames[i])
    do_request(contractAddresses[i], contractNames[i])

    files.download(contractNames[i] + '.json')